In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd 'drive/My Drive/pseudo-class generation'

import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

!nvidia-smi

Mounted at /content/drive
/content/drive/My Drive/pseudo-class generation
Thu Feb 29 18:32:05 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              11W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  

In [ ]:
pip install helpers

# Imports

In [ ]:
# License: BSD
# Author: Sasank Chilamkurthy

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True
plt.ion()   # interactive mode

from helpers import makedir
from log import create_logger
from preprocess import mean, std, preprocess_input_function

# Parameters

In [ ]:
# Models can be choosed from [resnet, alexnet, vgg, squeezenet, densenet, inception]
n_fold = '1'
model_name = "resnet"         # Resnet18
target_test_accu = 0.80

img_size = 300  # 224
num_epochs = 100
num_classes = 2

BATCH_SIZE = 80

train_batch_size = BATCH_SIZE
test_batch_size = BATCH_SIZE
train_push_batch_size = BATCH_SIZE
model_dir = './saved_models/' + model_name + '_fold'+ n_fold + '/'
makedir(model_dir)

# Load data

In [ ]:
magnification = '40x'
train_test = '/test'
fold = 'revised_fold' + n_fold + '_' + magnification
data_path = './dataset/' + fold + '/'
train_dir = data_path + 'train_push_balanced/train_augmented/'
test_dir = data_path + 'test/'
train_push_dir = data_path + 'train/'

log, logclose = create_logger(log_filename=os.path.join(model_dir, 'train.log'))

# load the data
normalize = transforms.Normalize(mean=mean,
                                 std=std)

# all datasets
# train set
train_dataset_0 = datasets.ImageFolder(
    train_dir,
    transforms.Compose([
        transforms.Resize(size=(img_size, img_size)),
        transforms.ToTensor(),
        normalize,
    ]))

#train_loader_0 = torch.utils.data.DataLoader(
#    train_dataset_0, batch_size=train_batch_size, shuffle=True,
#    num_workers=4, pin_memory=False)

# push set
train_push_dataset = datasets.ImageFolder(
    train_push_dir,
    transforms.Compose([
        transforms.Resize(size=(img_size, img_size)),
        transforms.ToTensor(),
        normalize,
    ]))
train_push_loader = torch.utils.data.DataLoader(
    train_push_dataset, batch_size=train_push_batch_size, shuffle=False,
    num_workers=4, pin_memory=False)

train_dataset = train_dataset_0 + train_push_dataset

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=train_batch_size, shuffle=True,
    num_workers=4, pin_memory=False)

# test set
test_dataset = datasets.ImageFolder(
    test_dir,
    transforms.Compose([
        transforms.Resize(size=(img_size, img_size)),
        transforms.ToTensor(),
        normalize,
    ]))
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=test_batch_size, shuffle=False,
    num_workers=4, pin_memory=False)

# we should look into distributed sampler more carefully at torch.utils.data.distributed.DistributedSampler(train_dataset)

#log('training_0 set size: {0}'.format(len(train_loader_0.dataset)))
log('training set size: {0}'.format(len(train_loader.dataset)))
log('push set size: {0}'.format(len(train_push_loader.dataset)))
log('test set size: {0}'.format(len(test_loader.dataset)))
log('batch size: {0}'.format(train_batch_size))

dataloaders = {'train' : train_loader , 'val': test_loader }
dataset_sizes = {'train' : len(train_loader) , 'val': len(test_loader) }

training set size: 10595
push set size: 995
test set size: 605
batch size: 80


# Model initialization

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
    if feature_extracting == False:
        for param in model.parameters():
            param.requires_grad = True

def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = img_size
        #model_ft.fc = nn.Identity()


    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = img_size

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg19(pretrained = use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = img_size

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = img_size

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        #model_ft.classifier = nn.Identity()
        input_size = img_size

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained = use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        # model_ft.fc = nn.Identity()
        # model_ft.AuxLogits.fc =  nn.Identity()
    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft

# training function

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, model_name = "inception"):
    since = time.time()

    if model_name == "inception":
       is_inception = True
    else:
       is_inception=False


    train_acc = []
    val_acc = []
    #train_loss = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 +  0.4 * loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            if phase == 'train':
               train_acc.append(epoch_acc)
               #train_loss.append(epoch_loss)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

            if phase == 'val':
                val_acc.append(epoch_acc)
                if best_acc > target_test_accu:
                    filepath = model_dir + str(epoch) + '_' +str(best_acc) + ".pt"
                    torch.save(model.state_dict(), filepath)

        print()

    torch.save(train_acc, 'Train_log')
    torch.save(val_acc, 'Test_log')
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    return model, train_acc, val_acc

#model = TheModelClass(*args, **kwargs)
#model.load_state_dict(torch.load(PATH))

def test(model , test_loader, num_Test_data, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    All_preds = []
    All_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels =  labels.to(device)
            preds = model(images)
            loss = criterion(preds , labels)
            test_loss += loss
            preds = torch.argmax(preds , dim = 1)
            All_preds.append(preds.cpu().numpy())
            All_labels.append(labels.cpu().numpy())

            correct += (preds == labels).float().sum()
        print(labels)
        print(preds)
        accuracy = correct / num_Test_data
        epoch_loss = test_loss / num_Test_data
        print("   loss:%.2f" %epoch_loss.item() , "  Acc:%.2f" %accuracy.item() )
        All_labels = np.concatenate( All_labels, axis=0 )
        All_preds = np.concatenate( All_preds, axis=0 )
    return All_labels ,All_preds

# Cunstruct the model

In [ ]:
criterion = nn.CrossEntropyLoss()

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = False

# Initialize the model for this run
modelA = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Send the model to GPU
modelA = modelA.to(device)
optimizerA = optim.Adam(modelA.parameters(), lr=0.001)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 185MB/s]


In [ ]:
# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = modelA.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in modelA.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in modelA.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

## Find total parameters and trainable parameters
#total_params = sum(p.numel() for p in modelA.parameters())
#print(f'{total_params:,} total parameters.')
#total_trainable_params = sum(
#    p.numel() for p in modelA.parameters() if p.requires_grad)
#print(f'{total_trainable_params:,} training parameters.')

# Training

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataset = test_dataset
val_loader = torch.utils.data.DataLoader(dataset = val_dataset, batch_size=BATCH_SIZE, shuffle=True)
dataloaders_dict = {'train' : train_loader , 'val': val_loader }
# Train and evaluate
modelA, train_acc, val_acc = train_model(modelA, dataloaders_dict, criterion, optimizerA, num_epochs = num_epochs , model_name = model_name)



# Compute Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedShuffleSplit

## Select trained model at epoch with best test/val accuracy as feature extractor
filepath2 = model_dir + "50_tensor(0.8793, device=_cuda_0_, dtype=torch.float64).pt"
model = modelA
model.load_state_dict(torch.load(filepath2))

Test_data = test_dataset
num_Test_data = len(Test_data)
test_loader = torch.utils.data.DataLoader(dataset = Test_data , batch_size=BATCH_SIZE, shuffle=True) # + val_dataset
All_labels ,All_preds = test(model, test_loader, num_Test_data, criterion)
print(confusion_matrix(All_labels ,All_preds ))
print(classification_report(All_labels ,All_preds ))

tensor([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1,
        1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1],
       device='cuda:0')
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       device='cuda:0')
   loss:0.01   Acc:0.81
[[132 107]
 [  5 361]]
              precision    recall  f1-score   support

           0       0.96      0.55      0.70       239
           1       0.77      0.99      0.87       366

    accuracy                           0.81       605
   macro avg       0.87      0.77      0.78       605
weighted avg       0.85      0.81      0.80       605

